In [1]:
import numpy as np
import os, sys
import pandas as pd
import glob
import re
import natsort
import sklearn
import datetime
import cv2
import shutil
import time
import h5py

from osgeo import gdal
from osgeo import gdal_array
from natsort import natsorted
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.insert(0, "/home/jgovoort/SRC")

In [4]:
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that checks and creates folder
from mkdir import check_create_dir
# Import functions for plots
from plots import plot_loss, plot_pred_test, plot_pred_train, plot_loss_multirun
# Import functions for metrics to monitoring accuracy
from metrics import coeff_determination
# Import functions for display
from display import Norma_Xpercentile
# Import functions for exportation of results
from export_results import save_predictions, write_run_metrics_file
# Import LRFinder function
from LRFinder import LRFinder

In [5]:
f = h5py.File('/home/jgovoort/Images/big_raster/tr_BNIR_patch/tr_Ymodel_S1BNIR.hdf5', 'r')
S1_patch_id = np.array(f["id_sar"])
S2_patch_id = np.array(f["id_rgb"])
S1_patch_lulc = np.array(f["lulc_sar"])
S2_patch_lulc = np.array(f["lulc_rgb"])
S1_patch_path = f["patch_sar_path"]
S2_patch_path = f["patch_rgb_path"]

In [6]:
lulc = S1_patch_lulc.reshape(S1_patch_lulc.size,1)
patch_id = S1_patch_id.reshape(S1_patch_lulc.size,1)

In [7]:


kf_SEED = 101
kf = KFold(n_splits=10, random_state=kf_SEED, shuffle=True)
idx_train = []
idx_val = []
for train_index, val_index in kf.split(np.zeros(len(lulc)),lulc):
    idx_train.append(train_index)
    idx_val.append(val_index)
print(len(idx_train))
print([len(x) for x in idx_train])
print(len(idx_val))
print([len(x) for x in idx_val])

10
[6229, 6229, 6230, 6230, 6230, 6230, 6230, 6230, 6230, 6230]
10
[693, 693, 692, 692, 692, 692, 692, 692, 692, 692]


In [8]:


kfold_nairobi_id = []
for i in range(len(idx_train)):
    kfold_nairobi_id.append((i+1,[x[0] for x in patch_id[idx_train[i]].tolist()],[x[0] for x in patch_id[idx_val[i]].tolist()]))



In [9]:
path_csv_kfold_split = '/home/jgovoort/Nairobi_10Fold_Split.csv'
a = pd.DataFrame(kfold_nairobi_id, columns=['kfold','ID_train','ID_val'])
a.to_csv(path_csv_kfold_split, index=False)
a

,kfold,ID_train,ID_val
0,1,"[10255, 10508, 10509, 10763, 10767, 11016, 110...","[16578, 17596, 18101, 18601, 18603, 19113, 203..."
1,2,"[10508, 10509, 10763, 10767, 11016, 11017, 112...","[10255, 17321, 17338, 17850, 18855, 19112, 193..."
2,3,"[10255, 10508, 10509, 10763, 10767, 11016, 112...","[11017, 15571, 16579, 16834, 17088, 17574, 183..."
3,4,"[10255, 10508, 10509, 10763, 10767, 11016, 110...","[14525, 18349, 18612, 19110, 19365, 19613, 196..."
4,5,"[10255, 10508, 10509, 10763, 10767, 11016, 110...","[11523, 16580, 16835, 17087, 18348, 18350, 183..."
5,6,"[10255, 10508, 10509, 10763, 11016, 11017, 112...","[10767, 11270, 11776, 16305, 17084, 17322, 180..."
6,7,"[10255, 10508, 10509, 10763, 10767, 11016, 110...","[16832, 17576, 17595, 17597, 18079, 19372, 206..."
7,8,"[10255, 10508, 10767, 11017, 11270, 11275, 115...","[10509, 10763, 11016, 12533, 14023, 15572, 168..."
8,9,"[10255, 10508, 10509, 10763, 10767, 11016, 110...","[11275, 12795, 14021, 14022, 16052, 16831, 170..."
9,10,"[10255, 10509, 10763, 10767, 11016, 11017, 112...","[10508, 11781, 14272, 16053, 17341, 17843, 180..."


In [ ]:
import os, signal

os.kill(os.getpid() , signal.SIGKILL)